In [5]:
from alive_progress import alive_it # For printing of progress in for loops
from pcapreader import pcapToDf
import os
import pandas as pd
from tabulate import tabulate


In [6]:
def refineLatexTableString(latexTableString):
    firstStart = latexTableString.find('{')
    formatStart = latexTableString.find('{',firstStart+1)
    firstEnd = latexTableString.find('}')
    formatEnd = latexTableString.find('}',firstEnd+1)
    replaceString = ''.join('|{}|'.format(x) for x in latexTableString[formatStart+1:formatEnd]).replace('||','|')
    latexTableString = latexTableString.replace(latexTableString[formatStart+1:formatEnd],replaceString,len(replaceString))
    newLineCounts = latexTableString.count("\n") - 1
    latexTableString = latexTableString.replace('\n','\n \hline ',newLineCounts)
    return latexTableString

In [7]:

## Get All pcap filenames:
victimFilenames = []
attackerFilenames = []
for root, dirs, files in os.walk("./"):
    for file in files:
        if file.endswith(".pcapng"):
            if "victim" in file:
                victimFilenames.append(os.path.join(root, file))
            if "attacker" in file:
                attackerFilenames.append(os.path.join(root, file))



## Get attacker bytes sent
attackerBytes = []
for attackFile in alive_it(attackerFilenames):
    # Get attack level
    start = attackFile.find('level')
    end = start+len('level')+1
    level = attackFile[start:end]
    attackerDf = pcapToDf(attackFile)
    # By finding the TFTP packets and summing the byte lengths we get the total number of bytes send by the attacker
    attackerDf = attackerDf.loc[attackerDf['Protocol'].isin(["TFTP"])]
    attackerBytesSent = attackerDf["Length"].sum()
    attackerBytes.append({'Level':level,'Attacker Bytes Sent':attackerBytesSent})

attackerBytes = pd.DataFrame(attackerBytes).sort_values('Level')

victimBytes = []
for victimFile in alive_it(victimFilenames):
    # Get attack level
    start = victimFile.find('level')
    end = start+len('level')+1
    level = victimFile[start:end]
    victimDf = pcapToDf(victimFile)
    # In the victim pcap filtering by destination port 50040 (the tftp servers source port) gives the tftp data transfered to the victim
    victimBytesReceived = victimDf.loc[(victimDf['Protocol'] == "UDP") & (victimDf['UDP Destination Port'] == 50040)]["Length"].sum()
    victimBytes.append({'Level':level,'Victim Bytes Received':victimBytesReceived})
victimBytes = pd.DataFrame(victimBytes).sort_values('Level')
extracted_col = attackerBytes['Attacker Bytes Sent']
byteStatisticTable = victimBytes.join(extracted_col)
byteStatisticTable['Amplification factor'] = byteStatisticTable['Victim Bytes Received']/byteStatisticTable['Attacker Bytes Sent']



|████████████████████████████████████████| 6/6 [100%] in 6:19.9 (0.02/s) 
|████████████████████████████████████████| 6/6 [100%] in 9.6s (0.63/s) 


In [9]:
print(tabulate(byteStatisticTable,headers='keys',tablefmt='fancy_grid',showindex=False))
latexTableString = byteStatisticTable.style.to_latex() # For getting the table into the report
latexTableString = refineLatexTableString(latexTableString)

print(latexTableString)

╒═════════╤═════════════════════════╤═══════════════════════╤════════════════════════╕
│ Level   │   Victim Bytes Received │   Attacker Bytes Sent │   Amplification factor │
╞═════════╪═════════════════════════╪═══════════════════════╪════════════════════════╡
│ level0  │                   48104 │                   840 │             57.2667    │
├─────────┼─────────────────────────┼───────────────────────┼────────────────────────┤
│ level1  │                  508528 │                  8880 │             57.2667    │
├─────────┼─────────────────────────┼───────────────────────┼────────────────────────┤
│ level2  │                 1813712 │                 80640 │             22.4915    │
├─────────┼─────────────────────────┼───────────────────────┼────────────────────────┤
│ level3  │                 2701042 │                781080 │              3.45809   │
├─────────┼─────────────────────────┼───────────────────────┼────────────────────────┤
│ level4  │                 3171468 │      